In [3]:
import pandas as pd
import numpy as np
import sys, os
import time
import io
import re
import datetime

In [4]:
### Changable parameters ### Remember to change tilt and packing density
samPath = '/Users/jacky/Documents/py3samsdk-master'
path_to_csv = 'csvs/'

year = 2014
description = 'flush' # set a description of of this data set (e.g. flush, tilted)
add_to_existing = True # if True, will add on to existing hourly.csv and yearly.csv files
############################
# Remember to change tilt and packing density below!

In [5]:
if samPath not in sys.path:
    sys.path.insert(0, samPath)
from py3samsdk.sscapi import PySSC
from urllib.error import HTTPError
from urllib.request import urlopen

ssc_lib = '/Applications/sdk-release/osx64/'  # path to SAM SSC Library
ssc = PySSC(ssc_lib)

In [6]:
zipcodes = pd.read_csv("sunroof_data/project-sunroof-postal_code-11292017.csv").loc[:,['region_name', 'state_name',  'lat_avg', 'percent_covered', 'lng_avg', 'number_of_panels_f', 'number_of_panels_total', 'yearly_sunlight_kwh_f']]
zipcodes['capacity'] = zipcodes['number_of_panels_f']*0.00025 # Google assumes each panel is 250W and we want capacity in MW
zipcodes['region_name'] = zipcodes['region_name'].apply(lambda x: str(x)[:-2])

# Set number to threshold that we want for percent_covered
zipcodes = zipcodes[zipcodes['percent_covered'] >= 80]
zipcodes = zipcodes[zipcodes['state_name'] != 'Alaska'] # NSRDB does not have weather data for most of Alaska
latlon = zipcodes.reset_index()
latlon.head()

,index,region_name,state_name,lat_avg,percent_covered,lng_avg,number_of_panels_f,number_of_panels_total,yearly_sunlight_kwh_f,capacity
0,1,15104,Pennsylvania,40.406255,98.791687,-79.862353,42634,130282.0,1.202862e+07,10.65850
1,3,15108,Pennsylvania,40.505561,96.886620,-80.187328,397143,863308.0,1.132624e+08,99.28575
2,4,15106,Pennsylvania,40.404535,99.683730,-80.094418,133591,350858.0,3.820641e+07,33.39775
3,5,15112,Pennsylvania,40.404671,99.732083,-79.839290,12877,44229.0,3.605938e+06,3.21925
4,6,15110,Pennsylvania,40.370982,99.574633,-79.852884,38592,73782.0,1.103744e+07,9.64800


In [7]:
tilt_size_buckets = pd.read_csv("sunroof_data/average_tilt_and_install_size_kw_buckets.csv")
tilt_size_buckets['zipcode'] = tilt_size_buckets['zipcode'].astype(str)
tilt_size_buckets = tilt_size_buckets.set_index('zipcode')
tilt_size_buckets.head()

,degrees_tilt_n,degrees_tilt_e,degrees_tilt_s,degrees_tilt_w,degrees_tilt_f,install_size_kw_buckets_json,install_size_kw_buckets_n_json,install_size_kw_buckets_e_json,install_size_kw_buckets_s_json,install_size_kw_buckets_w_json,install_size_kw_buckets_f_json
zipcode,,,,,,,,,,,
1001,18.057999,26.270523,27.476409,25.391170,2.931321,"[[0,1134],[5,1720],[10,774],[15,334],[20,136],...","[[0,283],[5,51],[10,12],[15,4],[20,3],[25,3],[...","[[0,1200],[5,597],[10,163],[15,44],[20,28],[25...","[[0,1389],[5,1195],[10,243],[15,45],[20,32],[2...","[[0,1045],[5,521],[10,142],[15,36],[20,30],[25...","[[0,212],[5,77],[10,33],[15,25],[20,19],[25,15..."
1002,17.444525,25.881517,28.781245,26.197519,2.940489,"[[0,550],[5,552],[10,265],[15,146],[20,95],[25...","[[0,123],[5,33],[10,10],[15,9],[20,3],[25,5],[...","[[0,463],[5,201],[10,62],[15,23],[20,19],[25,1...","[[0,562],[5,336],[10,134],[15,62],[20,34],[25,...","[[0,483],[5,206],[10,75],[15,40],[20,22],[25,1...","[[0,117],[5,51],[10,16],[15,12],[20,16],[25,17..."
1003,17.607019,24.261329,31.431343,28.438918,1.980439,"[[0,5],[5,4],[10,2],[15,6],[20,2],[25,4],[30,1...","[[0,4],[5,2],[15,1],[25,1],[35,1],[40,1],[45,1]]","[[0,8],[5,8],[10,6],[15,6],[20,2],[25,1],[30,1...","[[0,11],[5,7],[10,6],[15,8],[20,2],[25,3],[30,...","[[0,6],[5,2],[10,3],[15,5],[20,3],[25,6],[30,2...","[[0,8],[5,4],[10,2],[15,1],[20,2],[25,4],[30,2..."
1007,NaN,NaN,23.394337,24.708416,NaN,"[[0,1],[20,1],[25,1]]",[],[],"[[20,1],[25,1]]","[[0,1]]",[]
1013,18.209458,27.390362,27.410950,26.328913,3.701934,"[[0,1501],[5,2401],[10,982],[15,360],[20,145],...","[[0,406],[5,87],[10,16],[15,1],[20,3],[25,3],[...","[[0,1724],[5,763],[10,114],[15,35],[20,8],[25,...","[[0,1884],[5,1491],[10,270],[15,45],[20,11],[2...","[[0,1381],[5,658],[10,129],[15,31],[20,9],[25,...","[[0,316],[5,175],[10,72],[15,50],[20,30],[25,1..."


In [8]:
# info should describe the data (e.g. tilted roof, flat roof)
def create_hourly_csv(df, year, info):
    if info != '':
        info = '_' + info
    if not os.path.isfile('csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info)):
        df.to_csv('csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info))
    else:
        df.to_csv('csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info), mode='a', header=False)
    print("Saved hourly csv")
    
def create_yearly_csv(df, year, info):
    if info != '':
        info = '_' + info
#     if not os.path.isfile('csvs/agg_years/yearly{year}{info}.csv'.format(year = year, info = info)):
    df.to_csv('csvs/agg_years/yearly{year}{info}.csv'.format(year = year, info = info))
    print("Saved yearly csv")

In [9]:
def instantiate_tables():
    interval = '60'
    if description != '':
        info = '_' + description
    path_to_yearly = 'csvs/agg_years/yearly{year}{info}.csv'.format(year = year, info = info)
    path_to_hourly = 'csvs/agg_years/hourly{year}{info}.csv'.format(year = year, info = info)

    hourly_df = pd.DataFrame(index = pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
    hourly_df.columns.name = None
    hourly_df_zipcodes = []
    if os.path.isfile(path_to_yearly) and os.path.isfile(path_to_hourly):
        print("Reading existing files for {year}{info}".format(year = year, info = info))
        yearly_generations = pd.read_csv(path_to_yearly).set_index('region_name')['generations']
        yearly_generations.index = yearly_generations.index.astype(str) # zipcode indices are interpreted as int, but we want str
        hourly_df_zipcodes = pd.read_csv(path_to_hourly, index_col=0).index.astype(str)
        
    else:
        print("New tables will be generated")
        yearly_generations = pd.Series(np.nan, index = latlon['region_name'])
        
    return yearly_generations, hourly_df, hourly_df_zipcodes

In [14]:

def run_sam(year, direction):

    interval = '60'

    # Run weather data through SAM
    for index, row in latlon.iterrows():
        if row['region_name'] not in tilt_size_buckets.index:
            print("No tilt data available.")
            continue
        else:
            tilt = 15
            gcr = 0.7
        if not np.isnan(yearly_generations.loc[row['region_name']]) and row['region_name'] in existing_zipcodes_hourly:
            continue

        if os.path.isfile('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name'])):
            data = pd.read_csv('{path}{year}/{region}_{state}.csv'.format(path=path_to_csv, year=year, region=row['region_name'], state=row['state_name']))
        else:
            print("No weather data for " + row['region_name'])
            continue

        lat = row['lat_avg']
        lon = row['lng_avg']
        capacity = row['capacity']
        metadata = data.iloc[0:1, :]

        timezone = metadata['Time Zone'].values[0]
        elevation = metadata['Elevation'].values[0]

        # omit metadata at the top (hence the 2:)
        loc_data = data.iloc[:,:]
        loc_data.columns = loc_data.iloc[1] # reassign column names to the ones in the first row of the old table
        loc_data = loc_data.iloc[2:, :]
        loc_data = loc_data.set_index(pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
        loc_data = loc_data.dropna(axis = 1, how='all')
        loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']] = loc_data[['DNI','DHI', 'Wind Speed', 'Temperature']].apply(pd.to_numeric)

        wfd = ssc.data_create()
        ssc.data_set_number(wfd, 'lat', lat)
        ssc.data_set_number(wfd, 'lon', lon)
        ssc.data_set_number(wfd, 'tz', float(timezone))
        ssc.data_set_number(wfd, 'elev', float(elevation))
        ssc.data_set_array(wfd, 'year', loc_data.index.year)
        ssc.data_set_array(wfd, 'month', loc_data.index.month)
        ssc.data_set_array(wfd, 'day', loc_data.index.day)
        ssc.data_set_array(wfd, 'hour', loc_data.index.hour)
        ssc.data_set_array(wfd, 'minute', loc_data.index.minute)
        ssc.data_set_array(wfd, 'dn', loc_data['DNI'])
        ssc.data_set_array(wfd, 'df', loc_data['DHI'])
        ssc.data_set_array(wfd, 'wspd', loc_data['Wind Speed'])
        ssc.data_set_array(wfd, 'tdry', loc_data['Temperature'])

        # Create SAM compliant object  
        dat = ssc.data_create()
        ssc.data_set_table(dat, 'solar_resource_data', wfd)
        ssc.data_free(wfd)

        # Specify the system Configuration
        # Set system capacity in MW
        system_capacity = capacity
        ssc.data_set_number(dat, 'system_capacity', system_capacity)
        # Set DC/AC ratio (or power ratio). See https://sam.nrel.gov/sites/default/files/content/virtual_conf_july_2013/07-sam-virtual-conference-2013-woodcock.pdf
        ssc.data_set_number(dat, 'dc_ac_ratio', 1.15)
        # Set tilt of system in degrees
        # For Google data, roof segments are considered Flat for roofs with a tilt of less than 10%
        ssc.data_set_number(dat, 'tilt', tilt) ####tilt
        # Set azimuth angle (in degrees) from north (0 degrees)
        ssc.data_set_number(dat, 'azimuth', 180)
        # Set the inverter efficency
        ssc.data_set_number(dat, 'inv_eff', 96)
        # Set the system losses, in percent
        ssc.data_set_number(dat, 'losses', 14.0757)
        # Specify fixed tilt system (0=Fixed, 1=Fixed Roof, 2=1 Axis Tracker, 3=Backtracted, 4=2 Axis Tracker)
        ssc.data_set_number(dat, 'array_type', 0)
        # Set ground coverage ratio (PACKING DENSITY)
        ssc.data_set_number(dat, 'gcr', gcr)
#         ssc.data_set_number(dat, 'gcr', (np.cos(np.radians(lat)))**2) ####Packing Density
        # Set constant loss adjustment
        ssc.data_set_number(dat, 'adjust:constant', 0)

        # execute and put generation results back into dataframe
        mod = ssc.module_create('pvwattsv5')
        ssc.module_exec(mod, dat)
        loc_data['generation'] = np.array(ssc.data_get_array(dat, 'gen'))
        hourly_df.loc[:, row['region_name']] = loc_data['generation'] # column of generation data from SAM model
        # free the memory
        ssc.data_free(dat)
        ssc.module_free(mod)
        sys.stdout.write('\r{0}. {1}: {2}'.format(index, row['region_name'], loc_data['generation'].sum()))
        sys.stdout.flush()
        yearly_generations[row['region_name']] = loc_data['generation'].sum()*1000
        
        if (index + 1) % 1000 == 0:
            return index + 1
#             output_yearly = latlon.set_index('region_name')
#             output_yearly['generations'] = yearly_generations
#             print()
#             create_hourly_csv(hourly_df.T, year, description)
#             create_yearly_csv(output_yearly, year, description)
#             hourly_df = pd.DataFrame(index = pd.date_range('1/1/{yr}'.format(yr=year), freq=interval+'Min', periods=525600/int(interval)))
            
    return len(latlon)
    # latlon['generations'] = yearly_generations
    # latlon['error codes'] = errors

In [17]:
# Run this cell to start the program
for year in [2016]:
    for direction in ['n', 'w', 'e', 's']
        try:
            num_finished = 0
            while num_finished < len(zipcodes):
                yearly_generations, hourly_df, existing_zipcodes_hourly = instantiate_tables()
                num_finished = run_sam(year)
                output_yearly = latlon.set_index('region_name')
                output_yearly['generations'] = yearly_generations
                create_hourly_csv(hourly_df.T, year, description)
                create_yearly_csv(output_yearly, year, description)
        except (KeyboardInterrupt, Exception) as e:
            print()
            print(e)
            print("Closing. Saving current progress...")
            output_yearly = latlon.set_index('region_name')
            output_yearly['generations'] = yearly_generations


            create_hourly_csv(hourly_df.T, year, description)
            create_yearly_csv(output_yearly, year, description)
            print("Saved")
            pass


Reading existing files for 2012_flat_tilt_nrel
No tilt data available.
No tilt data available.
No weather data for 99258
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
Saved hourly csv
Saved yearly csv
Reading existing files for 2013_flat_tilt_nrel
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
Saved hourly csv
Saved yearly csv
Reading existing files for 2014_flat_tilt_nrel
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data available.
No tilt data availabl